# Iceberg Classification Step 5: Model Monitoring

Note that this notebook monitors the inference request while inference requests are made to the model served.

This notebook is tested with the following ``configuration`` from hopsworks.
<div>
<img src="fig/step4_5_jupyter_config.png" width="900" align="center"/>
</div>

In [1]:
import hops

# SparkSession available as 'spark'
print(
    f"-----------------------------------------------\n" \
    f"This notebook is tested with:\n" \
    f"  - Hopsworks {hops.__version__}.\n" \
    f"  - Spark {spark.version}.\n"
)

Starting Spark application


ID,YARN Application ID,Kind,State,Spark UI,Driver log
261,application_1619040920875_0294,pyspark,idle,Link,Link


SparkSession available as 'spark'.
-----------------------------------------------
This notebook is tested with:
  - Hopsworks 2.1.0.1.
  - Spark 2.4.3.2.

In [2]:
from hops import kafka
from hops import tls
from confluent_kafka import Producer, Consumer
# TOPIC_NAME can be found in the Kafka tab under hopsworks UI.
TOPIC_NAME = 'ShipIcebergClassifier-inf2529'

config = {
    "bootstrap.servers": kafka.get_broker_endpoints(),
    "security.protocol": kafka.get_security_protocol(),
    "ssl.ca.location": tls.get_ca_chain_location(),
    "ssl.certificate.location": tls.get_client_certificate_location(),
    "ssl.key.location": tls.get_client_key_location(),
    "group.id": "something"
}

consumer = Consumer(config)

def print_assignment(consumer, partitions):
    """ 
    Callback called when a Kafka consumer is assigned to a partition
    """
    print('Assignment:', partitions)
topics = [TOPIC_NAME]
consumer.subscribe(topics, on_assign=print_assignment)

In [3]:
for i in range(0, 1):
    msg = consumer.poll(timeout=30.0)
    if msg is not None:
        print('Consumed Message: {} from topic: {}, partition: {}, offset: {}, timestamp: {}'.format(msg.value(), msg.topic(), msg.partition(), msg.offset(), msg.timestamp()))
    else:
        print("Topic empty, timeout when trying to consume message, try to produce messages to the topic and then re-consume")

Assignment: [TopicPartition{topic=ShipIcebergClassifier-inf2529,partition=0,offset=-1001,error=None}]
Consumed Message: b'\n*ShipIcebergClassifier\x16\x82\x91\xc1\x9c\xb1^\x90\x03\xd0\xd2+{"instances": [[[[-33.6013298034668, -28.740568161010742, -31.170949935913086], [-32.906131744384766, -28.74061393737793, -30.823373794555664], [-32.262481689453125, -29.164443969726562, -30.713462829589844], [-32.906219482421875, -32.26252746582031, -32.584373474121094], [-29.61005973815918, -33.601505279541016, -31.60578155517578], [-27.580951690673828, -30.079727172851562, -28.830339431762695], [-27.950664520263672, -29.164621353149414, -28.55764389038086], [-27.950708389282227, -29.16466522216797, -28.55768585205078], [-26.88584327697754, -28.740922927856445, -27.813383102416992], [-28.740968704223633, -28.336898803710938, -28.5389347076416], [-28.336944580078125, -28.741012573242188, -28.538978576660156], [-28.741056442260742, -33.6018180847168, -31.171436309814453], [-30.080036163330078, -32.906

# End of Step 5.